In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
%matplotlib inline
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split 

In [2]:
df = pd.read_csv("Pima_Indian_diabetes.csv")

In [3]:
df.describe()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,742.000000,752.000000,768.000000,746.000000,768.000000,757.000000,768.000000,749.000000,768.000000
mean,3.866601,119.966097,68.886078,20.309879,79.799479,31.711151,0.471876,33.761336,0.348958
std,3.479971,32.367659,19.427448,15.974523,115.244002,8.544789,0.331329,12.297409,0.476951
min,-5.412815,0.000000,-3.496455,-11.945520,0.000000,-16.288921,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.100000,0.243750,24.000000,0.000000
50%,3.000000,116.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.000000,80.000000,32.000000,127.250000,36.500000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
df = df.drop(columns = ['SkinThickness'])
features = ['Pregnancies','Glucose', 'BloodPressure', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

In [5]:
for feature in features:
    df[feature].fillna(value=df[feature].mean(), inplace=True)

In [6]:
zero_insulin_data = df[ df.Insulin == 0 ]
non_zero_insulin_data = df[ df.Insulin != 0]

train_X = non_zero_insulin_data['Glucose'].values.reshape(-1,1)
train_y = non_zero_insulin_data['Insulin'].values.reshape(-1,1)
val_X = zero_insulin_data['Glucose'].values.reshape(-1,1)

model = LinearRegression()
model.fit(train_X, train_y)
predicted_y = model.predict(val_X)

j = 0
for i in df.index:
    if df.at[i, 'Insulin'] == 0:
        df.at[i, 'Insulin'] = predicted_y[j][0]
        j+=1


In [7]:
df.describe()

,Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.866601,119.966097,68.886078,152.778203,31.711151,0.471876,33.761336,0.348958
std,3.420480,32.028277,19.427448,98.668180,8.483295,0.331329,12.144139,0.476951
min,-5.412815,0.000000,-3.496455,-109.896240,-16.288921,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,90.000000,27.275000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,135.000000,32.000000,0.372500,30.000000,0.000000
75%,6.000000,139.000000,80.000000,190.075164,36.425000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [8]:
for feature in features:
    df.loc[df[feature] < 0, feature] = 0

In [9]:
df.describe()

,Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.885689,119.966097,68.895389,153.368229,31.760002,0.471876,33.761336,0.348958
std,3.384891,32.028277,19.393614,97.423387,8.261584,0.331329,12.144139,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,90.000000,27.275000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,135.000000,32.000000,0.372500,30.000000,0.000000
75%,6.000000,139.000000,80.000000,190.075164,36.425000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [10]:
df.loc[df.Insulin >250, 'Insulin'] = 250

In [11]:
df.isnull().describe()

,Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768,768,768,768,768,768,768,768
unique,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False
freq,768,768,768,768,768,768,768,768


In [12]:
accuracy = []

for i in range(100):
    df = df.sample(frac=1)
    X = df[df.columns[0:7]]
    y = df[df.columns[7]] 
    
    trainX, valX, trainY, valY = train_test_split(X, y, test_size = 0.20)
    lr = LogisticRegression(max_iter=2000, solver='lbfgs')
    lr.fit(trainX, trainY)
    accuracy.append(lr.score(valX, valY)*100)

In [13]:
average_accuracy = sum(accuracy)/len(accuracy)

In [14]:
print(average_accuracy)

76.75974025974023
